# Projeto de Previsão de Falha

**Autor:** Lucas Porto  
**Desafio:** Data Science - Meli  
**Objetivo:** Documentar a arquitetura e regras de negócio do sistema de previsão de falhas


---

## Visão Geral do Projeto

### Desafio proposto

#### Descrição
Os armazéns no mercado livre possuem uma frota de máquinas que transmitem diariamente medidas sobre seu status e funcionamento.
As técnicas de manutenção preditiva são projetadas para ajudar a determinar a condição do equipamento de manutenção em serviço para prever quando a manutenção deve ser realizada.
Esta abordagem promete economia de custos em relação à manutenção preventiva de rotina ou baseada no tempo, porque as tarefas são executadas apenas quando justificada E m média uma interrupção por falha custa 4x a mais do que uma interrupção preventiva.
O arquivo "full_devices.csv" possui os valores diários de 9 atributos dos dispositivos ao longo do tempo e a coluna que você está tentando prever é chamada de 'failure' com o valor binário 0 para nenhuma falha e 1 para falha, a coluna do dispositivo possui o id do dispositivo.


### Entregável
O objetivo é gerar um notebook Jupyter com um modelo para prever a falha do dispositivo antes de uma possível falha. Tente calcular e maximizar a possível economia gerada pelo modelo.


### Realizado

Para este desafio proposto iremos criar uma regra de negócios para calcularmos o melhor custo benefício entre o custo entre uma falha não detectada e uma manuntenção desnecessária caso prevêssemos errado. Iremos explicar melhor na Regra de Negócio.

## Imports

In [ ]:
import pandas as pd


from utils.data_utils import (
    load_and_explore_data,
    analyze_data_completeness,
    analyze_failure_patterns,
    verify_data_leakage,
)

from utils.feature_utils import (
    create_features,
)

from utils.model_utils import (
    temporal_cv_evaluation,
    create_pipeline_with_metadata,
)

from utils.pipeline_utils import (
    tune_neg_per_pos,
    train_evaluate_with_presplit,
    apply_optimal_thresholds,
    create_final_summary,
    save_model_and_metrics,
    analyze_feature_importance,
    calculate_event_metrics,
    select_best_model,
    execute_final_model_evaluation
)

In [ ]:
HORIZON_DAYS = 10
TAXA_ALERTAS = 0.05
C_FN = 100_000
C_FP = 25_000 
COST_RATIO = C_FN / C_FP
MIN_PREC_FOR_ROI = C_FP / (C_FP + C_FN)

## Regras de Negócio

### Estrutura de Custos

A regra de negócio central é baseada na **análise de custo-benefício**:

```python
# Constantes de custo definidas no projeto
C_FN = 100_000    # Custo de uma falha não detectada (False Negative)
C_FP = 25_000     # Custo de manutenção desnecessária (False Positive)
```

### Cálculo de ROI

**ROI = (Benefícios - Custos) / Custos × 100%**

Onde:
- **Benefícios:** Falhas evitadas × Custo da falha
- **Custos:** Manutenções desnecessárias × Custo da manutenção

### Precisão Mínima e colocamos como se fosse Break-even do projeto

```python
def min_precision_for_roi(c_fn=100_000, c_fp=25_000):
    """
    Calcula a precisão mínima necessária para atingir break-even
    Fórmula: C_FP / (C_FP + C_FN) = 25k / 125k = 20%
    """
    return c_fp / (c_fp + c_fn)  # = 0.20 (20%)
```

### Ratio de Custos
```python
COST_RATIO = C_FN / C_FP  # = 4.0
# Uma falha não detectada custa 4x mais que uma manutenção desnecessária
```

### Lógica de Decisão

O sistema otimiza para:
1. **Maximizar detecção de falhas** (True Positives)
2. **Minimizar manutenções desnecessárias** (False Positives)
3. **Respeitar orçamento de alertas** (evitar muitos alertas)



## Carga dos Dados e Análise

In [ ]:
# 1. Carregar e explorar dados
df = load_and_explore_data()

In [ ]:
df.info()

### Análise de completude dos dados

In [ ]:

# 2. Analisar completude dos dados
completeness_info = analyze_data_completeness(df)

### Análise dos padrões de falhas

In [ ]:
# 3. Analisar padres de falhas
device_failures, daily_failures = analyze_failure_patterns(df)

## Feature Engineering

### Tipos de Features Criadas

- **Features Básicas**
- Atributos originais (attribute1-9)
- Identificação do dispositivo
- Timestamp da observação
- **Features Temporais**
- **Features de Contexto**
- **Features de Interação**
- **Normalização por Dispositivo**


### Resultado Final
- **198 features** criadas
- **80 features** selecionadas pelo pipeline
- **Normalização por dispositivo** para lidar com heterogeneidade


### Criação de nova target

A nova target foi pensada em nos gerar um alerta que permite detectar a falha antes que ela aconteça. Que é o objetivo do nosso trabalho.

> "Se um dispositivo falha no dia D, então nos dias D-1,..., D-10 ele deveria ter recebido um alerta."

Criamos ele com a função abaixo, isso foi pensado em criamos um alerta de até 10 dias antes do ocorrido.

```python
def make_early_warning_labels_safe(df, horizon_days=10):
```

In [ ]:
# 4. Feature Engineering +  Target 
df_features, new_target_col = create_features(
    df,
    target_col="failure",
    forecast_horizon=HORIZON_DAYS,
    enable_interactions=True,
    max_lags=7,
)

In [ ]:

# 5.  Verificação de data leakage
suspicious_features = verify_data_leakage(df_features, new_target_col)

## Pipeline com Metadados

Quando trabalhamos com dados temporais, precisamos garantir que:
- **Metadados temporais** (datas, dispositivos) sejam preservados
- **Split temporal** seja feito antes do processamento
- **Features** sejam selecionadas apenas no treino
- **Validação** respeite a ordem cronológica


1. **Evita Data Leakage**
   - Split temporal antes do processamento
   - Features selecionadas apenas no treino
   - Metadados preservados para validação

2. **Preserva Contexto Temporal**
   - Datas mantidas para análise temporal
   - Dispositivos identificados para cooldown
   - Ordem cronológica respeitada

3. **Facilita Validação**
   - Métricas por evento
   - Análise de cooldown
   - ROI baseado em falhas reais


Separamos os dados temporais em:
- **Dados de treino**
- **Dados de validação:** separado para otimizaro o modelo
- **Dados de validação final (holdout)**: utilizado para validar sem vazamento
- **Dados de teste**: para nossa aplicação final.

In [ ]:
# 6.  Pipeline com metadados
(
    X_train,
    y_train,
    X_val_model,
    y_val_model,
    X_val_hold,
    y_val_hold,
    X_test,
    y_test,
    selected_features,
    dates_train,
    devices_train,
    dates_val_model,
    devices_val_model,
    dates_val_hold,
    devices_val_hold,
    dates_test,
    devices_test,
) = create_pipeline_with_metadata(df_features, target_col=new_target_col)

## Validação Cruzada Temporal

A **Validação Cruzada Temporal** é uma técnica que simula o cenário real de produção, onde:
- **Dados futuros** não estão disponíveis no treinamento
- **Ordem cronológica** deve ser respeitada
- **Performance** deve ser consistente ao longo do tempo

### Implementação de proteção

1. **Purge de 7 dias** entre treino e validação
2. **Split temporal rigoroso** (não aleatório)
3. **Pipeline aplicado** em cada fold
4. **Métricas consistentes** entre folds

### Sinais de Alerta

- **Custo crescente** ao longo dos folds
- **Recall decrescente** no tempo
- **Alta variabilidade** entre folds
- **Performance ruim** em folds recentes

In [ ]:
# 7. CV Ttemporal
cv_output = temporal_cv_evaluation(
    df_features,
    target_col=new_target_col,
    n_splits=3,
    purge_days=7,
)
cv_results = cv_output["cv_results"]
cv_summary = cv_output["cv_summary"]

Com base nos nossos dados, temos um modelo com custo aumentando com o tempo, variabilizade alta, mas o recall se mantem um valor razoável.

Podemos ver que temos drift nos dados e alta variabilidade no tempo devido nosso custo aumentar.

## Otimizando o Balanceamento

### Criação de parâmetro para otimização

O **neg_per_pos** é um parâmetro criado que define quantos **exemplos negativos** (não-falha) usar para cada **exemplo positivo** (falha) durante o treinamento:

```python
# Exemplo:
# neg_per_pos = 8 significa:
# - 1 exemplo de falha (positivo)
# - 8 exemplos de não-falha (negativos)
# - Ratio: 1:8 (12.5% de falhas)
```

### Por Que Precisamos de Tuning?

#### **1. Problema de Classes Desbalanceadas**
- **Dados originais**: 0.09% de falhas (106 falhas em 124,494 registros)
- **Após early warning**: ~1% de falhas (ainda muito desbalanceado)
- **Modelo precisa** de mais exemplos negativos para aprender

#### **2. Trade-off Crítico**
- **Muitos negativos** → Modelo fica conservador (baixo recall)
- **Poucos negativos** → Modelo fica agressivo (muitos falsos positivos)
- **Balanceamento ótimo** → Performance máxima


### Critérios de Seleção

#### **1. Custo de Negócio**
- **Custo total** = 4 × FN + 1 × FP
- **Objetivo**: Minimizar custo
- **Prioridade**: Evitar falhas não detectadas

#### **2. Taxa de Alertas**
- **Máximo**: 5% dos dispositivos
- **Objetivo**: Não sobrecarregar operações
- **Balanceamento**: Performance vs. Operacional

#### **3. Métricas Técnicas**
- **Precision**: > 20% (break-even)
- **Recall**: > 60% (cobertura de falhas)
- **F1-Score**: Balanceamento geral



In [ ]:
# 8. Otimização e Balanceamento
best_us = tune_neg_per_pos(
    X_train,
    y_train,
    X_val_model,
    y_val_model,
    candidates=(6, 8, 10, 12, 15, 20),
    max_alert_rate=TAXA_ALERTAS,
)

Dentro dos parâmetros que selecionamos, tivemos o melhor tunning com: 
- Distribuição: Negativos x Positivos em 8

## Treinando os Modelos

### **Modelos Selecionados**
- XGBoost: XGBClassifier
- LGBM: LGBMClassifier
- Ensemble entre ambos

### **Otimização de Thresholds**

Utilizamos dentro do treinamento a seleção do melhor threshold.


| Métrica Técnica | Métrica de Negócio | Implementação |
|-----------------|-------------------|----------------|
| **AUC-ROC** | Capacidade de discriminação | `roc_auc_score()` |
| **Average Precision** | Performance em desbalanceado | `average_precision_score()` |
| **Confusion Matrix** | **Custo (4×FN + 1×FP)** | `4 * fn + fp` |
| **Threshold** | **Orçamento de alertas** | `tune_alert_budget_simple()` |
| **Precision** | **Break-even (20%)** | `min_precision=0.20` |

In [ ]:
# 9. Treinar e avaliar com conjuntos pr-split
models_results = train_evaluate_with_presplit(
    X_train,
    y_train,
    X_val_model,
    y_val_model,
    X_test,
    y_test,
    min_pr=MIN_PREC_FOR_ROI,
    X_val_hold=X_val_hold,
    y_val_hold=y_val_hold,
    dates_train=dates_train,
    devices_train=devices_train,
    dates_val=dates_val_model,
    devices_val=devices_val_model,
    dates_test=dates_test,
    devices_test=devices_test,
    random_state=42,
    apply_balance=True,
    neg_per_pos=best_us["neg_per_pos"],
    max_neg_cap=50_000,
    calibrate=True,
    use_alert_budget=True,
    alert_budgets=(0.01, 0.015, 0.02, 0.03),
    feature_names=selected_features,
    selected_features=selected_features,
)


## Aplicação dos Thresholds

A função `apply_optimal_thresholds` é a **otimização final** do sistema que:

1. **Testa múltiplos orçamentos** de alertas (0.2% a 5%)
2. **Encontra thresholds ótimos** para cada orçamento
3. **Avalia performance** com métricas de negócio
4. **Escolhe a melhor configuração** para produção

### Processo de Otimização

#### Teste em orçamentos
```python
for budget in budgets:
    # Encontrar threshold que respeita o orçamento
    threshold = find_threshold_for_budget(
        y_val, 
        model_proba_val,
        target_budget=budget
    )
    
    # Aplicar threshold no teste
    y_pred_test = (model_proba_test >= threshold).astype(int)
    
    # Calcular métricas
    cost = calculate_business_cost(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test)
    recall = recall_score(y_test, y_pred_test)
    alert_rate = y_pred_test.mean()
    
    # Verificar restrições
    if precision >= min_precision and alert_rate <= budget:
        results[budget] = {
            'threshold': threshold,
            'cost': cost,
            'precision': precision,
            'recall': recall,
            'alert_rate': alert_rate
        }
```

#### Seleção da melhor configuração
```python
# Escolher orçamento com menor custo
best_budget = min(results.keys(), key=lambda x: results[x]['cost'])

# Configuração final
best_config = {
    'budget': best_budget,
    'threshold': results[best_budget]['threshold'],
    'cost': results[best_budget]['cost'],
    'precision': results[best_budget]['precision'],
    'recall': results[best_budget]['recall'],
    'alert_rate': results[best_budget]['alert_rate']
}
```


In [ ]:
# 10. Aplicação dos thresholds ótimos com orçamento de alertas
models_results = apply_optimal_thresholds(
    models_results,
    y_val_model, 
    y_test,
    dates_val=dates_val_model,
    devices_val=devices_val_model,
    dates_test=dates_test,
    devices_test=devices_test,
    df_features=df,  
    min_precision=MIN_PREC_FOR_ROI, 
    c_fn=C_FN,
    c_fp=C_FP,
    budgets=(
        0.002,
        0.003,
        0.004,
        0.005,
        0.006,
        0.008,
        0.010,
        0.012,
        0.015,
        0.020,
        0.030,
        0.050,
    ), 
    horizon_days=HORIZON_DAYS, 
)

## Feature Importance

Analisamos o feature importance para cada modelo, para analisarmos quais são mais improtantes.

In [ ]:
# 11. Analisar feature importance
models_results = analyze_feature_importance(models_results, selected_features)

## Calculando métricas por evento

A métrica por evento é calculada para ser mais um dos parametros usados na seleção do nosso modelo.

In [ ]:
# 12. Métricas por evento
models_results = calculate_event_metrics(
    models_results, df, dates_test, devices_test
)

In [ ]:

# 13. Selecionar o melhor modelo usando holdout e métrica de custo
best_model_name, best_model_results, comparison_df = select_best_model(models_results, y_val_hold)


In [ ]:
# 14. Execução final do melhor modelo com dados de teste
final_test_results = execute_final_model_evaluation(
    best_model_name=best_model_name,
    best_model_results=best_model_results,
    X_test=X_test,
    y_test=y_test,
    dates_test=dates_test,
    devices_test=devices_test,
    df=df,
    c_fn=C_FN,
    c_fp=C_FP,
    horizon_days=HORIZON_DAYS,
)

In [ ]:
# 15. Resumo final e métricas de ROI
summary = create_final_summary(
    models_results,
    dates_train,
    dates_val_model,
    dates_val_hold,
    dates_test,
    selected_features,
    cv_results,
    cv_summary,
    suspicious_features,
    c_fn=C_FN,
    c_fp=C_FP,
    min_roi=MIN_PREC_FOR_ROI
)

save_paths = save_model_and_metrics(
    best_model_name=best_model_name,
    best_model_results=best_model_results,
    selected_features=selected_features,
    summary=summary,
    final_test_results=final_test_results,
    cv_results=cv_results,
    cv_summary=cv_summary,
    suspicious_features=suspicious_features,
    model_name="device_failure_model",
    save_dir="models",
)